In [2]:
from __future__ import print_function
import torch as t
from torch.autograd import Variable as V

In [10]:
a = V(t.ones(3, 4), requires_grad=True)
print(a)

b = V(t.zeros(3, 4))
print(b)

c = a.add(b)
print(c)

d = c.sum()

d.backward()

print(c.data.sum(), c.sum())

print(a.grad)

print(a.requires_grad, b.requires_grad, c.requires_grad)
print(a.is_leaf, b.is_leaf, c.is_leaf)
print(c.grad)


tensor([[1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.]], requires_grad=True)
tensor([[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]])
tensor([[1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.]], grad_fn=<AddBackward0>)
tensor(12.) tensor(12., grad_fn=<SumBackward0>)
tensor([[1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.]])
True False True
True True False
None
tensor([[1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.]])


In [13]:
def f(x):
    y = x**2 * t.exp(x)
    return y

def gradf(x):
    dx = 2*x*t.exp(x) + x**2*t.exp(x)
    return dx

x = V(t.randn(3, 4), requires_grad=True)
y = f(x)

print(y)

y.backward(t.ones(y.size()))
print(x.grad)
print(gradf(x))

tensor([[ 0.1425,  0.0671,  1.4004,  7.8635],
        [ 0.5099, 13.2147,  0.5409,  9.2657],
        [ 0.1612,  5.6834,  0.5097,  0.2148]], grad_fn=<MulBackward0>)
tensor([[-4.5144e-01,  6.4828e-01,  4.9226e+00,  1.9133e+01],
        [-1.4816e-01,  2.9545e+01,  1.5267e-02,  2.1924e+01],
        [ 1.1125e+00,  1.4662e+01,  1.0712e-01, -4.5915e-01]])
tensor([[-4.5144e-01,  6.4828e-01,  4.9226e+00,  1.9133e+01],
        [-1.4816e-01,  2.9545e+01,  1.5267e-02,  2.1924e+01],
        [ 1.1125e+00,  1.4662e+01,  1.0712e-01, -4.5915e-01]],
       grad_fn=<AddBackward0>)


In [29]:
x = V(t.ones(1))
b = V(t.rand(1), requires_grad=True)
w = V(t.rand(1), requires_grad=True)
y = w*x
z = y + b
print(x.requires_grad, b.requires_grad, w.requires_grad, y.requires_grad)
print(x.is_leaf, w.is_leaf, b.is_leaf, y.is_leaf, z.is_leaf)
print(z.grad_fn)
print(z.grad_fn.next_functions)
print(z.grad_fn.next_functions[0][0] == y.grad_fn)
print(y.grad_fn.next_functions)
print(w.grad_fn, x.grad_fn)

z.backward(retain_graph=True)
print(w.grad)

z.backward()
print(w.grad)

False True True True
True True True False False
((<MulBackward0 object at 0x110fd0bd0>, 0L), (<AccumulateGrad object at 0x110fd0b50>, 0L))
True
((<AccumulateGrad object at 0x110fd0b90>, 0L), (None, 0L))
None None
tensor([1.])
tensor([2.])


In [35]:
def abs(x):
    
    if x.data[0] > 0: return x
    else: return -x
    
x = V(t.ones(1), requires_grad=True)
y = abs(x)
y.backward()
print(x.grad)
y.backward()
print(x.grad)

x = V(-t.ones(1), requires_grad=True)
y = abs(x)
y.backward()
print(x.grad)

tensor([1.])
tensor([2.])
tensor([-1.])


In [43]:
def f(x):
    result = 1
    for ii in x:
        if ii.item() > 0: result = ii * result
        
    return result

x = V(t.arange(-2.0, 4.0), requires_grad=True)
y = f(x)
y.backward()

print(x.grad)

tensor([0., 0., 0., 6., 3., 2.])


In [45]:
x = V(t.ones(1))
w = V(t.rand(1), requires_grad=True)
y = x * w
print(x.requires_grad, w.requires_grad, y.requires_grad)

False True True


In [49]:
with t.no_grad():
    x = V(t.ones(1))
    w = V(t.rand(1), requires_grad=True)
    y = w*x
print(x.requires_grad, w.requires_grad, y.requires_grad)

False True False


In [4]:
x = V(t.ones(3), requires_grad=True)
w = V(t.rand(3), requires_grad=True)
y = x*w
z = y.sum()

print(x.requires_grad, w.requires_grad, y.requires_grad)

z.backward()

print(x.grad, w.grad, y.grad)

#autograd.grad

x = V(t.ones(3), requires_grad=True)
w = V(t.rand(3), requires_grad=True)
y = x*w
z = y.sum()

print(x.requires_grad, w.requires_grad, y.requires_grad)

z.backward()

print(t.autograd.grad(z, y))
print(x.grad, w.grad, y.grad)

    

True True True
tensor([0.0872, 0.7775, 0.3234]) tensor([1., 1., 1.]) None
True True True
(tensor([1., 1., 1.]),)
tensor([0.3811, 0.7363, 0.0251]) tensor([1., 1., 1.]) None


In [7]:
#hook

def variable_hook(grad):
    print(grad)
    
x = V(t.ones(3), requires_grad=True)
w = V(t.rand(3), requires_grad=True)


y = x * w

#print(x.shape, w.shape, y.shape)
#print(x, w, y)

hook_handle = y.register_hook(variable_hook)
z = y.sum()
z.backward()
hook_handle.remove()

tensor([1., 1., 1.])


In [11]:
x = V(t.arange(0.0, 3.0), requires_grad=True)
y = x**2 + x*2
z = y.sum()
z.backward()
print(x.grad)

x = V(t.arange(0.0, 3.0), requires_grad=True)
y = x**2 + x*2
z = y.sum()
y_grad_variables = V(t.Tensor([1, 1, 1]))
y.backward(y_grad_variables)
print(x.grad)


tensor([2., 4., 6.])
tensor([2., 4., 6.])
